<a href="https://colab.research.google.com/github/anacasicande/Modelamiento/blob/main/aire_india.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd  # Importa la librería pandas para el manejo de datos
from datetime import datetime  # Importa la clase datetime para trabajar con fechas
import matplotlib.pyplot as plt  # Importa la función pyplot de matplotlib para graficar

# Leer el archivo CSV con pandas
df = pd.read_csv('city_day.csv')  # Carga el archivo CSV en un DataFrame

# Asegurarse de que la columna de fecha sea de tipo datetime
df['Date'] = pd.to_datetime(df['Date'])  # Convierte la columna 'Date' a tipo datetime

# Filtrar las filas que corresponden al año de interés
ano_interes = 2020  # Define el año de interés
df['Year'] = df['Date'].dt.year  # Crea una nueva columna 'Year' que contiene solo el año de la fecha

# Filtrar solo los datos del año de interés y asegurarse de que NOx sea numérico
df_ano_interes = df[(df['Year'] == ano_interes) & (df['NOx'].notna())]  # Filtra el DataFrame para el año de interés y valores no nulos de NOx

# Eliminar duplicados por fecha para obtener un solo dato al día
df_ano_interes = df_ano_interes.drop_duplicates(subset='Date')  # Elimina fechas duplicadas

# Convertir los datos de NOx a lista y las fechas correspondientes
datos = df_ano_interes['NOx'].astype(float).tolist()  # Convierte los valores de NOx a una lista de floats
fechas = df_ano_interes['Date'].tolist()  # Convierte las fechas a una lista

# Función para calcular derivada hacia adelante
def derivada_hacia_adelante(datos, fechas):  # Define una función para calcular la derivada hacia adelante
    derivadas = []  # Inicializa una lista vacía para almacenar las derivadas
    fechas_derivadas = []  # Inicializa una lista vacía para almacenar las fechas de las derivadas
    for i in range(len(datos)-1):  # Itera sobre todos los datos menos el último
        h = (fechas[i+1] - fechas[i]).days  # Calcula la diferencia en días entre dos fechas consecutivas
        if h > 0:  # Si la diferencia en días es mayor que 0
            derivadas.append((datos[i+1] - datos[i]) / h)  # Calcula la derivada y la agrega a la lista
            fechas_derivadas.append(fechas[i])  # Agrega la fecha correspondiente a la lista
    return derivadas, fechas_derivadas  # Retorna las derivadas y las fechas correspondientes

# Función para calcular derivada hacia atrás
def derivada_hacia_atras(datos, fechas):  # Define una función para calcular la derivada hacia atrás
    derivadas = []  # Inicializa una lista vacía para almacenar las derivadas
    fechas_derivadas = []  # Inicializa una lista vacía para almacenar las fechas de las derivadas
    for i in range(1, len(datos)):  # Itera desde el segundo dato hasta el último
        h = (fechas[i] - fechas[i-1]).days  # Calcula la diferencia en días entre dos fechas consecutivas
        if h > 0:  # Si la diferencia en días es mayor que 0
            derivadas.append((datos[i] - datos[i-1]) / h)  # Calcula la derivada y la agrega a la lista
            fechas_derivadas.append(fechas[i])  # Agrega la fecha correspondiente a la lista
    return derivadas, fechas_derivadas  # Retorna las derivadas y las fechas correspondientes

# Función para calcular derivada centrada
def derivada_centrada(datos, fechas):  # Define una función para calcular la derivada centrada
    derivadas = []  # Inicializa una lista vacía para almacenar las derivadas
    fechas_derivadas = []  # Inicializa una lista vacía para almacenar las fechas de las derivadas
    for i in range(1, len(datos)-1):  # Itera desde el segundo dato hasta el penúltimo
        h = (fechas[i+1] - fechas[i-1]).days  # Calcula la diferencia en días entre dos fechas separadas
        if h > 0:  # Si la diferencia en días es mayor que 0
            derivadas.append((datos[i+1] - datos[i-1]) / (2 * h))  # Calcula la derivada centrada
            fechas_derivadas.append(fechas[i])  # Agrega la fecha correspondiente a la lista
    return derivadas, fechas_derivadas  # Retorna las derivadas y las fechas correspondientes

# Función para graficar las derivadas y guardarlas como PNG
def graficar(derivadas, tipo_derivada, fechas, nombre_archivo):  # Define la función para graficar y guardar el archivo
    plt.figure(figsize=(10, 6))  # Define el tamaño de la figura
    plt.title(f'Derivada {tipo_derivada} en función del tiempo')  # Establece el título del gráfico
    plt.xlabel('Fecha')  # Etiqueta del eje X
    plt.ylabel(f'Derivada {tipo_derivada}')  # Etiqueta del eje Y
    plt.plot(fechas, derivadas, marker='o', linestyle='-', label=tipo_derivada)  # Grafica los puntos con un marcador circular
    maximo = max(derivadas)  # Encuentra el valor máximo de la derivada
    max_index = derivadas.index(maximo)  # Encuentra el índice del valor máximo
    fecha_max = fechas[max_index]  # Encuentra la fecha correspondiente al máximo
    plt.text(fecha_max, maximo, f'Máximo: {round(maximo, 2)}\nFecha: {fecha_max.strftime("%Y-%m-%d")}',
             ha='left', va='bottom', rotation=20)  # Añade una anotación con el valor máximo y la fecha
    plt.ylim(min(derivadas) - 0.5, maximo + 0.5)  # Establece los límites del eje Y
    plt.grid(True)  # Muestra una cuadrícula
    plt.legend()  # Añade una leyenda
    plt.savefig(nombre_archivo)  # Guarda la gráfica como archivo PNG
    plt.close()  # Cierra la figura

# Función para analizar los máximos cambios y guardarlos en un archivo de texto
def analizar_variaciones(derivadas, fechas, tipo_derivada, archivo_texto):  # Define la función para analizar las variaciones
    maximo = max(derivadas)  # Encuentra el valor máximo de la derivada
    min_derivada = min(derivadas)  # Encuentra el valor mínimo de la derivada
    max_index = derivadas.index(maximo)  # Encuentra el índice del máximo
    min_index = derivadas.index(min_derivada)  # Encuentra el índice del mínimo

    fecha_max = fechas[max_index]  # Encuentra la fecha del valor máximo
    fecha_min = fechas[min_index]  # Encuentra la fecha del valor mínimo

    with open(archivo_texto, 'a') as f:  # Abre el archivo de texto en modo adjuntar
        f.write(f"\nAnálisis de la derivada {tipo_derivada}:\n")  # Escribe el tipo de derivada
        f.write(f"- Máximo cambio positivo en NOx: {round(maximo, 2)} el {fecha_max.strftime('%Y-%m-%d')}\n")  # Escribe el máximo cambio positivo
        f.write(f"- Máximo cambio negativo en NOx: {round(min_derivada, 2)} el {fecha_min.strftime('%Y-%m-%d')}\n")  # Escribe el máximo cambio negativo

# Calcular las derivadas
derivadas_adelante, fechas_adelante = derivada_hacia_adelante(datos, fechas)  # Calcula la derivada hacia adelante
derivadas_atras, fechas_atras = derivada_hacia_atras(datos, fechas)  # Calcula la derivada hacia atrás
derivadas_centrada, fechas_centrada = derivada_centrada(datos, fechas)  # Calcula la derivada centrada

# Graficar las derivadas y guardarlas como PNG
graficar(derivadas_adelante, "Hacia adelante", fechas_adelante, "derivada_adelante.png")  # Guarda la gráfica de la derivada hacia adelante
graficar(derivadas_atras, "Hacia atrás", fechas_atras, "derivada_atras.png")  # Guarda la gráfica de la derivada hacia atrás
graficar(derivadas_centrada, "Centrada", fechas_centrada, "derivada_centrada.png")  # Guarda la gráfica de la derivada centrada

# Analizar variaciones y guardar en archivo de texto
archivo_resultados = 'analisis_derivadas.txt'  # Define el nombre del archivo de texto para los resultados
analizar_variaciones(derivadas_adelante, fechas_adelante, "hacia adelante", archivo_resultados)  # Analiza las variaciones para la derivada hacia adelante
analizar_variaciones(derivadas_atras, fechas_atras, "hacia atrás", archivo_resultados)  # Analiza las variaciones para la derivada hacia atrás
analizar_variaciones(derivadas_centrada, fechas_centrada, "centrada", archivo_resultados)  # Analiza las variaciones para la derivada centrada

# Escribir observaciones generales en el archivo de texto
with open(archivo_resultados, 'a') as f:  # Abre el archivo de texto en modo adjuntar
    f.write("\nAl observar las diferentes gráficas se pudo evidenciar que, en el caso de la derivada hacia atrás y adelante, coinciden en su valor máximo, mientras que la centrada difiere considerablemente. Lo anterior se debe a que la derivada centrada es mejor cuando se tienen datos con un comportamiento suave, puesto que asume un comportamiento lineal entre i-1 e i+1. Lo anterior en este caso no se cumple, debido a que las mediciones de NOx varían considerablemente día a día. Es importante conocer los días en que se da la mayor variación, porque permite evidenciar de manera inmediata qué actividad se varía en esos días próximos que pudo generar un aumento brusco de las emisiones. Esto puede permitir promover políticas que tengan en cuenta las actividades con mayor impacto logrando así minimizar el efecto de estas en la salud de la población y demás aspectos que deben considerarse a nivel ambiental.\n")  # Añade el análisis solicitado al final del archivo de texto
